In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [4]:
train

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [5]:
train.shape

(25000, 3)

In [6]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [7]:
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords

nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /home/ios/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import re

def review_to_words( raw_review ):
    review_text = BeautifulSoup( raw_review ).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english")) # ??? why sets much faster than lists
    meaningful_words = [w for w in words if not w in stops]
    return(" ".join( meaningful_words ))




In [9]:
clean_review = review_to_words( train["review"][0] )

In [13]:
num_reviews = train["review"].size
clean_train_reviews = []

for i in range(0, num_reviews):
    clean_train_reviews.append( review_to_words( train["review"][i] ))

/tmp/ipykernel_583473/1830946893.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup( raw_review ).get_text()


In [15]:
len(clean_train_reviews)

25000

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word", tokenizer = None, preprocessor = None, stop_words = None, max_features=5000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

(25000, 5000)

In [24]:
vocab = vectorizer.get_feature_names_out()
vocab

array(['abandoned', 'abc', 'abilities', ..., 'zombie', 'zombies', 'zone'],
      dtype=object)

In [29]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train_data_features, train["sentiment"])

In [30]:
train

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."
